<h2>0. Data Generation</h2>
<h4>Genrate Synthetic Patient Data through Synthea</h4>

1. Go to link: <a>https://github.com/synthetichealth/synthea</a>, follow Developer Quick Start and install Synthea. </p>

2. Go to <span style="color:powderblue;">.../synthea/src/main/resources/synthea.properties</span>, change <span style="color:powderblue;">exporter.csv.export = true</span>. </p>

3. Open cmd, input <span style="color:powderblue;">cd .../synthea</span>, then <span style="color:powderblue;">run_synthea -s 1256 -p 5000 Massachusetts </span> (project 1). Then, input <span style="color:powderblue;">run_synthea -s 1256 -p 5000 California </span> (project 2 and 3). This generate 5000 patients' information located at Massachusetts/California with seed 1256.</p>

4. In .../synthea/output/csv, locate following files: patients.csv, medications.csv, conditions.csv.</p>




<h2>1. Data Localization</h2>
<h4>Change US info to Australia</h4>

Given that Synthea defaultly generates US address, Consider manually change the following columns for better data visualization: </p>
 1. ETHNICITY: From "hispanic" and "non-hispanic" to "Aboriginal and Torres Strait Islander" and "other". Change ratio of Aboriginal and Torres Strait Islander to 3.2% (based on the 2021 census). </p>
 
 2. CITY, STATE，COUNTY，ZIP，LAT，LON：The geomap mainly focues on Victoria region. Therefore STATE are all changed to Victoria, and COUNTY are randomly assign to any Victoria suburb. CITY, ZIP, LAT, LON asign according to the Database of Australian Postcodes: <a>https://www.matthewproctor.com/australian_postcodes</a>.Suburb name is based on VIC Local Government Areas GDA2020 (2nd intake) : <a>https://data.gov.au/dataset/ds-dga-bdf92691-c6fe-42b9-a0e2-a4cd716fa811/details</a></p>

 Reason why we leave BIRTHPLACE, ADDRESS，FIPS unchanged:</p>
Birthplace and address is highly variated. They are not likely to play a role in data visualization. FIPS is a unique geography identifier used by the US. This system does not apply to Austalia. Therefore, these are leaved unchanged.

In [1]:
import pandas as pd
import random
import geopandas as gpd
random.seed(1256)

#Import .shp file for suburb info
geo_state = gpd.read_file('data/sf/vic_localities.shp')

#Import genetrated Synthea data
patients_df = pd.read_csv('data/Synthetic data/project2and3/patients.csv')

1. ETHNICITY

In [2]:
#Generate ethnicities with the proportions
patients_df['ETHNICITY'] = random.choices(
    ["Other", "Aboriginal and Torres Strait Islander"],
    weights=[0.968, 0.032],  
    k=len(patients_df)
)

patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f9fed45c-a11b-60ce-21f5-18573f43e43d,2001-01-23,NaN,999-97-7295,S99931414,X4413788X,Mr.,Riley817,Gislason620,NaN,...,Hawaiian Gardens,California,Los Angeles County,6059.0,90630,33.797332,-118.077014,1764.16,85581.40,16045
1,6e99dcce-b24c-bf6f-8e4a-93bbcd286ec4,1983-01-03,NaN,999-21-3729,S99927332,X52800232X,Mr.,Jamey282,Pfannerstill264,NaN,...,Los Angeles,California,Los Angeles County,6037.0,90058,33.929979,-118.273823,23398.09,45693.92,24442
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2021-04-07,NaN,999-67-4381,NaN,NaN,NaN,Quinn173,Runolfsson901,NaN,...,Los Angeles,California,Los Angeles County,6037.0,91405,34.310508,-118.570361,5053.16,6286.68,66755
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,2000-03-22,NaN,999-10-2867,S99945644,X21338348X,Mr.,Carroll471,Reichert620,NaN,...,Los Angeles,California,Los Angeles County,6037.0,90031,34.207267,-118.333245,4750.00,54852.42,15662
4,42bf13d8-30b7-23dd-0bab-338a9e445071,2013-11-01,NaN,999-20-8605,NaN,NaN,NaN,Cornell131,Hoppe518,NaN,...,Yorba Linda,California,Orange County,6059.0,92870,33.848635,-117.814314,32442.56,19233.60,146580


 2. CITY, STATE，COUNTY，ZIP，LAT，LON：

In [3]:
#Change STATE to Victoria
patients_df['STATE'] = "Victoria"

#Get suburb name from .shp file and drop None
Victoria_suburb = geo_state['LOC_NAME'].tolist()

#Randomly assign Australia suburb to original dataframe 
patients_df["COUNTY"] = random.choices(Victoria_suburb, k=len(patients_df))
patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f9fed45c-a11b-60ce-21f5-18573f43e43d,2001-01-23,NaN,999-97-7295,S99931414,X4413788X,Mr.,Riley817,Gislason620,NaN,...,Hawaiian Gardens,Victoria,Double Bridges,6059.0,90630,33.797332,-118.077014,1764.16,85581.40,16045
1,6e99dcce-b24c-bf6f-8e4a-93bbcd286ec4,1983-01-03,NaN,999-21-3729,S99927332,X52800232X,Mr.,Jamey282,Pfannerstill264,NaN,...,Los Angeles,Victoria,Koondrook,6037.0,90058,33.929979,-118.273823,23398.09,45693.92,24442
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2021-04-07,NaN,999-67-4381,NaN,NaN,NaN,Quinn173,Runolfsson901,NaN,...,Los Angeles,Victoria,Pira,6037.0,91405,34.310508,-118.570361,5053.16,6286.68,66755
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,2000-03-22,NaN,999-10-2867,S99945644,X21338348X,Mr.,Carroll471,Reichert620,NaN,...,Los Angeles,Victoria,Lang Lang,6037.0,90031,34.207267,-118.333245,4750.00,54852.42,15662
4,42bf13d8-30b7-23dd-0bab-338a9e445071,2013-11-01,NaN,999-20-8605,NaN,NaN,NaN,Cornell131,Hoppe518,NaN,...,Yorba Linda,Victoria,Korweinguboora,6059.0,92870,33.848635,-117.814314,32442.56,19233.60,146580


To modify the ZIP, LAT, LON, we can refer to the Database of Australian Postcodes: <a>https://www.matthewproctor.com/australian_postcodes</a>

In [4]:
#Modify CITY, ZIP, LAT, LON accordingly
postcodes_df = pd.read_csv("data/australian_postcodes.csv")

# Map COUNTY to locality in postcodes_df to get corresponding ZIP, LAT, LON
county_to_zip_lat_lon = dict(zip(postcodes_df["locality"].str.title(), 
                                 postcodes_df[["postcode", "lat", "long", "lgaregion"]].values.tolist()))

In [5]:
# Ensure the mapping returns a tuple of (ZIP, LAT, LON, CITY) for every row, even if some mappings are None
patients_df['mapped'] = patients_df['COUNTY'].map(county_to_zip_lat_lon).apply(lambda x: x if isinstance(x, list) else [None, None, None, None])

# Now, separate this mapped data into individual columns
patients_df['ZIP'] = patients_df['mapped'].apply(lambda x: x[0] if x[0] is not None else 0).astype(int)
patients_df['LAT'] = patients_df['mapped'].apply(lambda x: x[1] if x[1] is not None else 0)
patients_df['LON'] = patients_df['mapped'].apply(lambda x: x[2] if x[2] is not None else 0)
patients_df['CITY'] = patients_df['mapped'].apply(lambda x: x[3] if x[3] is not None else 'Unknown')

# Drop the temporary 'mapped' column as it's no longer needed
patients_df.drop('mapped', axis=1, inplace=True)

patients_df.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,f9fed45c-a11b-60ce-21f5-18573f43e43d,2001-01-23,NaN,999-97-7295,S99931414,X4413788X,Mr.,Riley817,Gislason620,NaN,...,East Gippsland,Victoria,Double Bridges,6059.0,3893,-37.495349,147.866067,1764.16,85581.40,16045
1,6e99dcce-b24c-bf6f-8e4a-93bbcd286ec4,1983-01-03,NaN,999-21-3729,S99927332,X52800232X,Mr.,Jamey282,Pfannerstill264,NaN,...,Gannawarra,Victoria,Koondrook,6037.0,3580,-35.639761,144.094672,23398.09,45693.92,24442
2,8842ad59-b185-c6ce-871a-dcdcf111757a,2021-04-07,NaN,999-67-4381,NaN,NaN,NaN,Quinn173,Runolfsson901,NaN,...,Swan Hill,Victoria,Pira,6037.0,3585,-35.348388,143.336906,5053.16,6286.68,66755
3,d0c2d7e5-2e4a-5d6a-cd68-70824b671847,2000-03-22,NaN,999-10-2867,S99945644,X21338348X,Mr.,Carroll471,Reichert620,NaN,...,Bass Coast,Victoria,Lang Lang,6037.0,3984,-38.312647,145.563979,4750.00,54852.42,15662
4,42bf13d8-30b7-23dd-0bab-338a9e445071,2013-11-01,NaN,999-20-8605,NaN,NaN,NaN,Cornell131,Hoppe518,NaN,...,Hepburn,Victoria,Korweinguboora,6059.0,3461,-37.287848,144.170995,32442.56,19233.60,146580


In [6]:
#explore patients.csv
patients_df.to_csv("data/modified data/project2/patients.csv", index=False)

Confrim that the suburb name matches the shape file.

In [7]:
all_elements_present = patients_df['COUNTY'].isin(geo_state['LOC_NAME']).all()
print(all_elements_present)

True


For non-standard project (project 3), change the project 2 colunmn name from 'COUNTY' to 'SUBURB' and save to a different file.

In [8]:
# Rename the column from "COUNTY" to "SUBURB"
patients_df.rename(columns={'COUNTY': 'SUBURB'}, inplace=True)

# Save the DataFrame to a new CSV file
patients_df.to_csv("data/modified data/project3/patients.csv", index=False)